In [1]:
import fastbook
fastbook.setup_book()
from fastai.vision.all import *

In [18]:
# Test to view the behavior of view with -1
p = torch.randn(3, 4)
v = p.view(-1, 12)
p, v

(tensor([[-0.6890, -1.1267, -0.2858, -1.0935],
         [ 1.1351,  0.7592, -3.5945,  0.0192],
         [ 0.1052,  0.9603, -0.5672, -0.5706]]),
 tensor([[-0.6890, -1.1267, -0.2858, -1.0935,  1.1351,  0.7592, -3.5945,  0.0192,  0.1052,  0.9603, -0.5672, -0.5706]]))

In [4]:
Path = untar_data(URLs.MNIST_SAMPLE)

In [6]:
threes = (Path/'train'/'3')
sevens = (Path/'train'/'7')

In [7]:
stacked_threes = torch.stack([tensor(Image.open(o)) for o in threes.ls()])
stacked_sevens = torch.stack([tensor(Image.open(o)) for o in sevens.ls()])

In [8]:
stacked_threes.shape, stacked_sevens.shape

(torch.Size([6131, 28, 28]), torch.Size([6265, 28, 28]))

In [17]:
threes_and_sevens = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)
threes_and_sevens.shape
labels_threes_and_sevens = tensor([1]*len(stacked_threes) + [0]*len(stacked_sevens)).unsqueeze(1)
labels_threes_and_sevens.shape, threes_and_sevens.shape


(torch.Size([12396, 1]), torch.Size([12396, 784]))

In [22]:
# create a DataLoader and a dataset, dataloader further batches the data for each run
dset = list(zip(threes_and_sevens, labels_threes_and_sevens))
dl = DataLoader(dset, batch_size=64, shuffle=True)

In [26]:
# Initialize weights and biases
def init_weights(x, b):
    w = torch.randn(x).requires_grad_()
    bias = torch.randn(b).requires_grad_()
    return (w,bias)
weights, bias = init_weights(28*28, 1)
weights.shape, bias.shape


(torch.Size([784]), torch.Size([1]))

In [27]:
# define the model
def model(x, w, bias):
    return x@w + bias

In [28]:
def loss(preds, labels):
    return torch.where(labels == 1, 1-preds, preds).mean()

In [ ]:
def train_epoch(dl, w, b):
    for x, y in dl:
        preds = model(x, w, b)
        l = loss(preds, y)
        l.backward()
        with torch.no_grad():
            w -= w.grad * 0.01
            b -= b.grad * 0.01
            w.grad.zero_()
            b.grad.zero_()
    return preds

In [ ]:
dset_new = list(zip(threes_and_sevens[:4], labels_threes_and_sevens[:4]))
dl_new = DataLoader(dset_new, batch_size=64, shuffle=True) 
preds = model(dl_new.x, weights, bias)


TypeError: unsupported operand type(s) for @: 'tuple' and 'Tensor'